In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
plt.style.use('ggplot')
sns.set(font_scale=1.5)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
import os, pickle, re, string
from string import punctuation
from nltk.corpus import stopwords
# model imports
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.word2vec import Word2Vec
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# languange processing imports
import nltk
from gensim.corpora import Dictionary
from collections import Counter

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv("/content/split_dataset.csv", sep=',' )

In [ ]:
processed_Review = pd.DataFrame(df, columns = ['processed'])
processed_Review

,processed
0,تحميلك للتطبيق بالخطأ يدخلك عزل لمدة ١٤ يوم بش...
1,يعمل يستجيب مرحلة التسجيل لايتم ارسال الكود
2,للأسف تطبيق سيئ جدا بقالي يومين بحاول اسجل يرس...
3,قدرت اسجل التطبيق اوصل ل آخر خطوه وهي كود التح...
4,نزلت البرنامج اكثر مره وسجلت بياناتي كلها لاني...
...,...
7780,استطع تسجيل جديد ورطونا بالاجازات المرضيه اطبع...
7781,التطبيق مهم ياريت تربطون الدخول بنفس بيانات ال...
7782,تجربة سيئة وتطبيق غبي أستطيع عمل التسجيل الجديد
7783,مشاكل استطيع اختيار المركز ابي اطلع الاجازة قد...


In [ ]:
processed_Review['processed'] = processed_Review['processed'].astype(str)

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


data = processed_Review.values.tolist()
data_words = list(sent_to_words(data))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Create dictianry and corpus

In [ ]:
# use Gensim to create a dictionary 
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

print('Number of unique tokens: %d' % len(data_words))
print('Number of documents: %d' % len(corpus))


# View
print(corpus[:1][0][:30])

Number of unique tokens: 7785
Number of documents: 7785
[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1)]


In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           alpha='symmetric',
                                       chunksize=100,
                                       passes=1,
                                       per_word_topics=True,
                                           eta=0.91)
from pprint import pprint
# Print the Keyword in the  topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.019*"التسجيل" + 0.018*"التطبيق" + 0.013*"خطا" + 0.012*"البرنامج" + '
  '0.010*"يقول" + 0.009*"الدخول" + 0.008*"تسجيل" + 0.006*"الخادم" + '
  '0.006*"اسجل" + 0.006*"جديد"'),
 (1,
  '0.005*"موعد" + 0.003*"فقط" + 0.003*"اختيار" + 0.003*"تستطيع" + 0.003*"خدمة" '
  '+ 0.002*"وزارة" + 0.002*"الوقت" + 0.002*"المدينة" + 0.002*"مركز" + '
  '0.002*"يعطيك"'),
 (2,
  '0.012*"كلمة" + 0.009*"المرور" + 0.007*"الموقع" + 0.005*"يطلب" + '
  '0.005*"تحديد" + 0.003*"عدم" + 0.003*"زي" + 0.003*"المرضية" + '
  '0.003*"الاجازات" + 0.002*"السر"'),
 (3,
  '0.012*"الله" + 0.009*"نجمه" + 0.004*"راضي" + 0.003*"مو" + 0.003*"ليه" + '
  '0.003*"العظيم" + 0.002*"ونعم" + 0.002*"بالله" + 0.002*"الوكيل" + '
  '0.002*"حسبي"'),
 (4,
  '0.031*"جدا" + 0.024*"تطبيق" + 0.020*"التطبيق" + 0.012*"سيء" + 0.011*"يعمل" '
  '+ 0.011*"فاشل" + 0.010*"برنامج" + 0.009*"موعد" + 0.009*"سيي" + '
  '0.007*"البرنامج"')]


In [ ]:
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3374948428116665


# Bag of Words

In [ ]:
words = []

for i in processed_Review['processed']:
  words.append(i.split(' '))

In [ ]:
dictionary = gensim.corpora.Dictionary(words)

count = 0
for k, v in dictionary.iteritems():
  print(k, v)
  count += 1
  if count > 10:
    break

0 إصابتك
1 اشتباه
2 البرامج
3 باصابتك
4 بالخطأ
5 بشكل
6 تحميلك
7 تلقائي
8 حذفته
9 صحتي
10 عزل


In [ ]:
## Filter out tokens in the dictionary by their frequency.
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [ ]:
#Create Corpus -> term document frequency
bow_corpus = [dictionary.doc2bow(doc) for doc in words]
bow_corpus[7781]

[(24, 1),
 (140, 2),
 (159, 1),
 (161, 1),
 (200, 1),
 (344, 1),
 (354, 1),
 (554, 1),
 (561, 1),
 (672, 1)]

In [ ]:
bow_doc_7781 = bow_corpus[7781]

for i in range(len(bow_doc_7781)):
  print("Word {} (\"{}\") appears {} time.".format(bow_doc_7781[i][0], 
                                                   dictionary[bow_doc_7781[i][0]], bow_doc_7781[i][1]))

Word 24 ("التطبيق") appears 1 time.
Word 140 ("الدخول") appears 2 time.
Word 159 ("مشكلة") appears 1 time.
Word 161 ("يوجد") appears 1 time.
Word 200 ("تسجيل") appears 1 time.
Word 344 ("مستخدم") appears 1 time.
Word 354 ("بنفس") appears 1 time.
Word 554 ("مهم") appears 1 time.
Word 561 ("ياريت") appears 1 time.
Word 672 ("بيانات") appears 1 time.


In [ ]:
def document_to_bow(processed):
  bow_corpus = [dictionary.doc2bow(doc) for doc in words]
  processed['bow'] = list(map(lambda doc: bow_doc_7781 , processed_Review.processed))
    
document_to_bow(processed_Review)

# TF-IDF

In [ ]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [ ]:
corpus_tfidf = tfidf[bow_corpus]


In [ ]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.3722194561401363),
 (1, 0.2650684574335239),
 (2, 0.3238854836623631),
 (3, 0.3418369049976069),
 (4, 0.330763757046554),
 (5, 0.3844290820845282),
 (6, 0.29020558587261963),
 (7, 0.26153123977645804),
 (8, 0.39953580517948745)]


# Runing LDA using bag of words corpus

In [ ]:
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, alpha='symmetric', eta=0.91)

In [ ]:
for idx, topic in lda_model_bow.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.029*"تطبيق" + 0.020*"التسجيل" + 0.020*"فاشل" + 0.012*"البرنامج" + 0.010*"رقم" + 0.010*"موعد" + 0.010*"جدا" + 0.009*"خطأ" + 0.009*"التطبيق" + 0.009*"راضي"
Topic: 1 
Words: 0.029*"التحديث" + 0.029*"التطبيق" + 0.014*"التسجيل" + 0.013*"الله" + 0.012*"خطأ" + 0.011*"موعد" + 0.010*"يعمل" + 0.008*"يقبل" + 0.007*"انه" + 0.007*"الدخول"
Topic: 2 
Words: 0.060*"التطبيق" + 0.022*"البرنامج" + 0.013*"تطبيق" + 0.013*"الدخول" + 0.012*"يقول" + 0.011*"التسجيل" + 0.011*"خطأ" + 0.011*"الله" + 0.009*"تسجيل" + 0.009*"برنامج"
Topic: 3 
Words: 0.026*"و" + 0.019*"جدا" + 0.017*"يقول" + 0.017*"تطبيق" + 0.015*"الموقع" + 0.014*"التطبيق" + 0.009*"اسجل" + 0.009*"موعد" + 0.009*"ان" + 0.008*"كلمة"
Topic: 4 
Words: 0.049*"التطبيق" + 0.044*"جدا" + 0.034*"يعمل" + 0.030*"البرنامج" + 0.016*"التسجيل" + 0.012*"سيء" + 0.012*"و" + 0.011*"برنامج" + 0.011*"ممتاز" + 0.009*"تطبيق"


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_bow.log_perplexity(bow_corpus)) 


Perplexity:  -6.560098483950876


# Running LDA using tf-idf

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, alpha='symmetric', eta=0.91)


In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.020*"جدا" + 0.018*"فاشل" + 0.015*"سيء" + 0.014*"تطبيق" + 0.014*"التسجيل" + 0.012*"التطبيق" + 0.012*"جيد" + 0.012*"البرنامج" + 0.011*"يفتح" + 0.010*"يحتاج"
Topic: 1 
Words: 0.015*"البرنامج" + 0.013*"تطبيق" + 0.012*"برنامج" + 0.010*"التطبيق" + 0.009*"تحديث" + 0.009*"التحديث" + 0.006*"متاح" + 0.006*"التسجيل" + 0.006*"يوجد" + 0.006*"ليه"
Topic: 2 
Words: 0.016*"جدا" + 0.012*"ممتاز" + 0.011*"التسجيل" + 0.011*"التطبيق" + 0.010*"الدخول" + 0.008*"تسجيل" + 0.008*"خطأ" + 0.008*"موعد" + 0.007*"مو" + 0.007*"تطبيق"
Topic: 3 
Words: 0.044*"يعمل" + 0.029*"التطبيق" + 0.013*"الله" + 0.011*"سيئ" + 0.010*"البرنامج" + 0.009*"لايعمل" + 0.009*"تطبيق" + 0.008*"جدا" + 0.007*"مشكلة" + 0.006*"لماذا"
Topic: 4 
Words: 0.018*"يقبل" + 0.015*"التسجيل" + 0.013*"التطبيق" + 0.012*"و" + 0.009*"التحديث" + 0.008*"يقول" + 0.008*"خطأ" + 0.007*"يعلق" + 0.006*"البرنامج" + 0.006*"لايعمل"


In [ ]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=corpus_tfidf, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  nan


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tfidf.log_perplexity(corpus_tfidf)) 


Perplexity:  -7.389744103471655


# Performance evaluation by classifying sample document using Basic LDA

In [ ]:
processed_docs = df['stem_meaningful']
processed_docs[50]


"['يوجد', 'مشكله', 'بموضوع', 'الاستبيان', 'بالنسبه', 'لجهاز', 'اندرويد', 'التطبيق', 'بفتح', 'مباشره', 'صفحة', 'استبيان', 'اليوم', 'يوجد', 'اي', 'معلومات', 'او', 'استبيان', 'لتعبئة']"

In [ ]:
for index, score in sorted(lda_model_tfidf[corpus_tfidf[50]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5450183153152466	 
Topic: 0.029*"يعمل" + 0.024*"التطبيق" + 0.019*"البرنامج" + 0.015*"التحديث" + 0.013*"جدا" + 0.013*"جيد" + 0.009*"يفتح" + 0.009*"يشتغل" + 0.008*"فاشل" + 0.007*"لايعمل"

Score: 0.23717613518238068	 
Topic: 0.023*"التسجيل" + 0.020*"جدا" + 0.015*"يقبل" + 0.015*"سيء" + 0.013*"التطبيق" + 0.012*"يعمل" + 0.009*"البرنامج" + 0.009*"برنامج" + 0.008*"لايقبل" + 0.008*"خطأ"

Score: 0.19737644493579865	 
Topic: 0.017*"الله" + 0.011*"التطبيق" + 0.011*"تطبيق" + 0.008*"و" + 0.008*"يعمل" + 0.007*"جدا" + 0.006*"نجمه" + 0.006*"البرنامج" + 0.006*"مشكلة" + 0.005*"موعد"

Score: 0.01021482702344656	 
Topic: 0.014*"ممتاز" + 0.011*"التطبيق" + 0.010*"متاح" + 0.009*"خطأ" + 0.009*"برنامج" + 0.009*"يقول" + 0.009*"جدا" + 0.009*"فاشل" + 0.008*"التسجيل" + 0.008*"الدخول"

Score: 0.010214326903223991	 
Topic: 0.018*"تطبيق" + 0.013*"التطبيق" + 0.011*"يقول" + 0.009*"التسجيل" + 0.008*"و" + 0.007*"موعد" + 0.006*"تسجيل" + 0.006*"جديد" + 0.006*"الله" + 0.006*"حل"


# Testing each model accuracy on unseen reviews


In [ ]:
test_corpus = df.processed[50:70]
test_corpus

50    يوجد مشكله بموضوع الاستبيان بالنسبه لجهاز اندر...
51    السلام عليكم ورحمه الله سمحتو ياجماعه انا حملت...
52    مافيه شي مميز ولايقدم اي ارشادات ولافيه تسجيل ...
53    تسجيل الدخول يعطيني خطاء البيانات وعند التسجيل...
54                          معلق ارسال الاستبيان اليومي
55    عندي اشتراك بالتطبيق يفتح ورقم الاثبات والجوال...
56    فين الاستبيانات اذا التطبيق مو شغال نحط التقيي...
57    التطبيق مشكله فنيه يغلق نفسه اختيار الدوله يست...
58    التطبيق مخبط يطلب اسبيان تقدر تكتب اي ملاحظة ف...
59    التطبيق لايعمل انشاء حساب يرفض الدخول بسبب ه خ...
60                                       سيء يفتح تعليق
61      مو حلو مابي ادخل الهويه ومدري وش وش دا يع بمسحو
62                                      سيئ جدا جدا جدا
63                   تبديل التاريخ ميلادى هجرة مش موجود
64    سجلت التطبيق طلع حجر ١٤ يوم اني اعاني اي شي ار...
65    البرنامج يعلق حالة الاستبيان للمصاب الرجاء إصل...
66                                         التطبيق يعمل
67                                   خاص لدولة ا

In [ ]:
unseen_review = 'التطبيق يرفض تسجيل الدخول'
#Tokenize the unseen review into words
tokens = [word for word in unseen_review.split()]
bow_vector = dictionary.doc2bow((tokens))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.8370928764343262	 Topic: 0.029*"يعمل" + 0.024*"التطبيق" + 0.019*"البرنامج" + 0.015*"التحديث" + 0.013*"جدا"
Score: 0.04094498232007027	 Topic: 0.018*"تطبيق" + 0.013*"التطبيق" + 0.011*"يقول" + 0.009*"التسجيل" + 0.008*"و"
Score: 0.04081331938505173	 Topic: 0.023*"التسجيل" + 0.020*"جدا" + 0.015*"يقبل" + 0.015*"سيء" + 0.013*"التطبيق"
Score: 0.0408010296523571	 Topic: 0.014*"ممتاز" + 0.011*"التطبيق" + 0.010*"متاح" + 0.009*"خطأ" + 0.009*"برنامج"
Score: 0.04034784808754921	 Topic: 0.017*"الله" + 0.011*"التطبيق" + 0.011*"تطبيق" + 0.008*"و" + 0.008*"يعمل"


In [ ]:
unseen_review = 'االتطبيق مشكله فنيه يغلق نفسه  '
#Tokenize the unseen review into words
tokens = [word for word in unseen_review.split()]
bow_vector = dictionary.doc2bow((tokens))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.7970845699310303	 Topic: 0.023*"التسجيل" + 0.020*"جدا" + 0.015*"يقبل" + 0.015*"سيء" + 0.013*"التطبيق"
Score: 0.05105307325720787	 Topic: 0.029*"يعمل" + 0.024*"التطبيق" + 0.019*"البرنامج" + 0.015*"التحديث" + 0.013*"جدا"
Score: 0.05083850026130676	 Topic: 0.014*"ممتاز" + 0.011*"التطبيق" + 0.010*"متاح" + 0.009*"خطأ" + 0.009*"برنامج"
Score: 0.05061442404985428	 Topic: 0.017*"الله" + 0.011*"التطبيق" + 0.011*"تطبيق" + 0.008*"و" + 0.008*"يعمل"
Score: 0.050409458577632904	 Topic: 0.018*"تطبيق" + 0.013*"التطبيق" + 0.011*"يقول" + 0.009*"التسجيل" + 0.008*"و"


In [ ]:
unseen_review = 'تسجيل الدخول يعطيني خطاء البيانات وعند التسجيل'
#Tokenize the unseen review into words
tokens = [word for word in unseen_review.split()]
bow_vector = dictionary.doc2bow((tokens))

for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))

Score: 0.8971348404884338	 Topic: 0.014*"ممتاز" + 0.011*"التطبيق" + 0.010*"متاح" + 0.009*"خطأ" + 0.009*"برنامج"
Score: 0.02591741271317005	 Topic: 0.023*"التسجيل" + 0.020*"جدا" + 0.015*"يقبل" + 0.015*"سيء" + 0.013*"التطبيق"
Score: 0.02571820467710495	 Topic: 0.029*"يعمل" + 0.024*"التطبيق" + 0.019*"البرنامج" + 0.015*"التحديث" + 0.013*"جدا"
Score: 0.025672482326626778	 Topic: 0.018*"تطبيق" + 0.013*"التطبيق" + 0.011*"يقول" + 0.009*"التسجيل" + 0.008*"و"
Score: 0.02555706910789013	 Topic: 0.017*"الله" + 0.011*"التطبيق" + 0.011*"تطبيق" + 0.008*"و" + 0.008*"يعمل"
